# Random Forrest

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.signal import iirfilter, filtfilt
#from data_analyzer import DataAnalyzer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import dill

"""
Date: 2024-06-20 15:47:56,,,,,,
Calibration: XL_ODR: 6667Hz, XL_FS: 16g, GY_ODR: 6667Hz, GY_FS: 2000dps,,,
Time(s),Acceleration X (g),Acceleration Y (g),Acceleration Z (g),Angular Momentum X (dps),Angular Momentum Y (dps),Angular Momentum Z (dps)
0.000083,0.693359,-0.720215,0.14209,-3.540039,0.366211,-3.051758
0.00098,0.70752,-0.755371,0.086914,1.953125,0.732422,-1.159668
"""

#dill.load_session('rf_env.db')

In [ ]:
# Load training data
train_data = pd.read_csv(r"../measurement/processed_data/train_long.csv", header=2)
X_train = train_data[["Time(s)", "Acceleration X (g)", "Acceleration Y (g)", "Acceleration Z (g)", "Angular Momentum X (dps)", "Angular Momentum Y (dps)", "Angular Momentum Z (dps)"]]
y_train = train_data["Stand detected"]

sampling_frequency = 1 / (train_data["Time(s)"].diff().mean())
cutoff_frequency = 100

# Apply IIR filter to training data (excluding time series)
b, a = iirfilter(4, Wn=cutoff_frequency, fs=sampling_frequency, btype="low", ftype="butter")
X_train_filtered = X_train.copy()
X_train_filtered.iloc[:, 1:] = filtfilt(b, a, X_train.iloc[:, 1:], axis=0)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filtered.iloc[:, 1:])

# Increase the weight of specific features
feature_weights = np.array([1, 1, 1, 1, 1, 1])  # Higher weights for Acc X, Acc Y, and Angular Z
X_train_weighted = X_train_scaled * feature_weights

# Convert data to float32
X_train_weighted = X_train_weighted.astype(np.float32)

# Build RandomForest model
model = RandomForestClassifier(n_estimators=11, random_state=24, n_streams=1)
model.fit(X_train_weighted, y_train)

In [ ]:
# Load test data
test_data = pd.read_csv(r"../measurement/processed_data/test.csv", header=2)
#test_data = pd.read_csv(r"../measurement/processed_data/moving_all_around/random.csv", header=2)
X_test = test_data[["Time(s)", "Acceleration X (g)", "Acceleration Y (g)", "Acceleration Z (g)", "Angular Momentum X (dps)", "Angular Momentum Y (dps)", "Angular Momentum Z (dps)"]]

# Apply IIR filter to test data (excluding time series)
X_test_filtered = X_test.copy()
X_test_filtered.iloc[:, 1:] = filtfilt(b, a, X_test.iloc[:, 1:], axis=0)

# Standardize the test data
X_test_scaled = scaler.transform(X_test_filtered.iloc[:, 1:])

predictions = model.predict(X_test_scaled)
if False:
    predictions = pd.Series(predictions).rolling(window=25, min_periods=1).mean().round().astype(int)
test_data["predicted_heel_button"] = predictions

In [ ]:
# Plot Angular Momentum Z axis (filtered and scaled)
fig = go.Figure()
# [:, 5]
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=X_test_scaled[:, 0], mode="lines", name="AC X (Filtered & Scaled)"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=X_test_scaled[:, 1], mode="lines", name="AC Y (Filtered & Scaled)"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=X_test_scaled[:, 2], mode="lines", name="AC Z (Filtered & Scaled)"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=X_test_scaled[:, 3], mode="lines", name="AM X (Filtered & Scaled)"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=X_test_scaled[:, 4], mode="lines", name="AM Y (Filtered & Scaled)"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=X_test_scaled[:, 5], mode="lines", name="AM Z (Filtered & Scaled)"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=test_data["Heel Button"], mode="lines", name="Heel Button"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=test_data["Stand detected"], mode="lines", name="Analytical"))
fig.add_trace(go.Scatter(x=test_data["Time(s)"], y=test_data["predicted_heel_button"], mode="lines", name="Predicted Heel Button"))
fig.show()

In [ ]:
dill.dump_session('rf_env.db')